In [2]:
##my stuff
from imagery_psychophysics.src.stirling_maps import sparse_point_maps as spm
from imagery_psychophysics.src.model_z import noise_grid
from variational import *

##other people's stuff
import math
import copy
import numpy as np
import pandas as pd

from fractions import gcd
from PIL.Image import open
from PIL import Image
from scipy.misc import imresize
from os.path import join
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib.colors import LinearSegmentedColormap
from scipy.misc import comb as nCk
from time import time
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams
from theano.tensor.shared_randomstreams import RandomStreams
from theano import function, shared
from theano.tensor.extra_ops import repeat, to_one_hot
from matplotlib import pyplot as plt
from matplotlib.mlab import griddata
%matplotlib inline

#### An analysis of some old data.
This somehow broke when I updated theano/cuda. But see "development_11" for the model run and some results.

In [6]:
##grab the data from a previous training session
store = pd.io.pytables.HDFStore('/home/tnaselar/NAS/tnaselar/imagery_probe/analysis_results/all_subjects.h5')

In [7]:
modelDf = store['easy_format']

SkipTest: You are importing theano.sandbox.cuda. This is the old GPU back-end and is removed from Theano. Use Theano 0.9 to use it. Even better, transition to the new GPU back-end! See https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

In [ ]:
##btw, this is how you get nested attributes
def rgetattribute(obj, listOfAttributes):
    if type(listOfAttributes) is not list:
        listOfAttributes = [listOfAttributes]
    try:
        return getattr(obj, listOfAttributes[-1])
    except:
        return rgetattribute(getattr(obj,  listOfAttributes.pop(0)), listOfAttributes)


##grab the modeling result you want
def get_model_attribute(attributeString,df,shapeOfAttribute=None):
    '''
    get_model_attribute(attributeString,df,shapeOfAttribute=None)
     attributeString can be a list of attributes if the desired attribute is nested.
    '''
    n = len(df)
    if shapeOfAttribute is not None:
        dfshape = [n]+list(shapeOfAttribute)
        newArray = np.full(dfshape,0)
    else:
        newArray = [0]*n
    for idx,row in df.iterrows():
        copyAttributeString = copy.deepcopy(attributeString)
        newArray[idx] = rgetattribute(df['model'].iloc[idx],copyAttributeString)
    
    return newArray
        
##...if you do something with it that results in nice plottable scalar, add it to a new df
def make_new_df(newColName,newColData,df):
    try:
        newDf = df.drop('model', axis=1)
    except:
        newDf = df
    newDf[newColName] = newColData
    return newDf



#### Here's an example of how awesome python is.
This puts the "update_qPi" methods for each bestModel in a list, as well as the corresponding qZ.
Passes qZ as argument to each method. Puts all outputs in a list.

In [ ]:
allQZ = get_model_attribute('bestQZ', modelDf)

In [ ]:
flarg = get_model_attribute('update_qPi', modelDf)

In [ ]:
qPiList = []
for qZ,update_func in zip(allQZ,flarg):
    _,qPi = update_func(qZ)
    qPiList.append(qPi)

In [ ]:
qPiList

#### Study metrics

In [ ]:
metricsDf = make_new_df('percentCorrect',get_model_attribute('bestPercentCorrect', modelDf,shapeOfAttribute=(1,)),modelDf)
metricsDf = make_new_df('subject+target', metricsDf['subject']+metricsDf['target'], metricsDf)

In [ ]:
metricsDf

In [ ]:
import seaborn as sns

In [ ]:
sns.pointplot(x='state',y='percentCorrect',hue='target', data=metricsDf)

In [ ]:
sns.pointplot(x='state',y='percentCorrect',hue='subject+target', data=metricsDf, )
plt.legend(loc='lower left')

#### Entropy

In [ ]:
entropy = get_model_attribute('posteriorEntropy_history',modelDf)


In [ ]:
numIters = get_model_attribute('numIters', modelDf,shapeOfAttribute=(1,))

In [ ]:
finalEntropy = np.full(len(numIters), 0)
ii = 0
for ent,n in zip(entropy,numIters.squeeze()-1):
    finalEntropy[ii] = ent[n]
    ii += 1
metricsDf = make_new_df('entropy', finalEntropy,metricsDf)
metricsDf
    

In [ ]:
sns.pointplot(x='state',y='entropy',hue='subject', data=metricsDf)

In [ ]:
sns.pointplot(x='state',y='entropy', hue='subject+target', data=metricsDf)
plt.legend(loc='lower left')

In [ ]:
sns.pointplot(x='state',y='entropy',hue='target', data=metricsDf)

In [ ]:
img = metricsDf[metricsDf['state']=='img'].entropy.values
pcp = metricsDf[metricsDf['state']=='pcp'].entropy.values
dx = np.argsort(pcp)
plt.plot(pcp[dx],img[dx],'-.')
plt.plot(np.linspace(0,150),np.linspace(0,150),)
plt.axis('equal')

#### Noise parameters

In [ ]:
noiseParams = get_model_attribute('bestNoiseParam', modelDf,shapeOfAttribute=(2,))
metricsDf = make_new_df('forgetting', 1-noiseParams[:,0], metricsDf)
metricsDf = make_new_df('halucinating', noiseParams[:,1], metricsDf)
metricsDf

In [ ]:
sns.pointplot(x='state',y='forgetting',hue='subject', data=metricsDf)

In [ ]:
sns.pointplot(x='state',y='forgetting',hue='target', data=metricsDf)

In [ ]:
sns.pointplot(x='state',y='forgetting',hue='subject+target', data=metricsDf)
plt.legend(loc='lower left')

In [ ]:
sns.pointplot(x='state',y='halucinating',hue='subject+target', data=metricsDf)
plt.legend(loc='lower left')

#### Resolution and capacity

In [ ]:
loa = ['responses', 'Z', 'numPixels', 'D']
numpixels = get_model_attribute(loa, modelDf,shapeOfAttribute=(1,))
metricsDf = make_new_df('numpixels', numpixels, metricsDf)
loa = ['responses', 'Z', 'categoryPrior', 'numObjects', 'K']
numobjects = get_model_attribute(loa, modelDf,shapeOfAttribute=(1,))
metricsDf = make_new_df('numobjects', numobjects, metricsDf)

metricsDf

In [ ]:
sns.pointplot(x='state',y='numpixels',hue='subject+target', data=metricsDf)
plt.legend(loc='lower left')

In [ ]:
sns.pointplot(x='state',y='numobjects',hue='subject+target', data=metricsDf)
plt.legend(loc='lower left')